# save dynamic data

> Fill in a module description here

In [ ]:
#| default_exp dynamic

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
#| export
import inspect
import datetime
import os
import sys
import platform
from reproduce_work.core import read_base_config, requires_config, generate_config,reproduce_dir,update_watched_files,validate_base_config

VAR_REGISTRY = {}


Setting reproduce.work config dir to ./reproduce


In [3]:
#generate_config()

In [4]:
#| export
from pathlib import Path
import hashlib
import inspect
import re
import toml

#def update_registry(var_name, value):
    

def get_cell_index():
    """
    Get the current cell index in a Jupyter notebook environment.
    If not in Jupyter, return None.
    """
    try:
        # Execute JavaScript to get the current cell index
        get_ipython().run_cell_magic('javascript', '', 'IPython.notebook.kernel.execute(\'current_cell_index = \' + IPython.notebook.get_selected_index())')
        return current_cell_index
    except:
        return None
    
def check_for_defintion_in_context(function_name='save'):
    assert function_name in ['save', 'assign'], "function_name must be either 'save' or 'assign'"
    
    from IPython import get_ipython
    ip = get_ipython()

    # Check if in Jupyter environment
    if ip is None:
        
        #fill this in 
        pass

    else:
        # Get the input history
        #lineno = inspect.stack()[0].lineno
        raw_hist = ip.history_manager.input_hist_raw
        current_cell = raw_hist[-1]


        matches = re.findall(rf"{function_name}\((.+?),", current_cell)
                
        if matches:
            # save call
            defined_var = matches[0].strip()
            definition_cell_content = ''
            
            for prior_cell in raw_hist[-2::-1]:
                #print(prior_cell)
                if f'{defined_var} =' in prior_cell or f'{defined_var}=' in prior_cell:
                    definition_cell_content = prior_cell
                    break
            
            # find the line number of the where the variable was defined
            # Give a window of 5 lines around the definition call
            def_cell_lines = definition_cell_content.split('\n')
            if len(def_cell_lines)>0:
                lineno = None
                for line_num, line in enumerate(def_cell_lines):
                    if defined_var in line:
                        lineno = line_num
                        break
                if lineno:
                    definition_context = (
                        '\n'.join(def_cell_lines[max(0, lineno-5):lineno]) + 
                        '\n🚩' + def_cell_lines[lineno] + '\n' +
                        '\n'.join(def_cell_lines[lineno+1:min(len(def_cell_lines), lineno+5)])
                    )
                else:
                    definition_context = None

            else:
                definition_context = None

            
            save_cell_lines = current_cell.split('\n')
            if len(save_cell_lines)>0:
                save_lineno = None
                for line_num, line in enumerate(save_cell_lines):
                    if 'save(' in line:
                        save_lineno = line_num
                        break
                
                if save_lineno:
                    save_context = (
                        '\n'.join(save_cell_lines[max(0, save_lineno-5):save_lineno]) + 
                        '\n🚩' + save_cell_lines[save_lineno] + '\n' +
                        '\n'.join(save_cell_lines[save_lineno+1:min(len(save_cell_lines), save_lineno+5)])
                    )
                else:
                    save_context = None
                
            else:
                save_context = None
            

        else:
            # not a save call
            save_context = None
            definition_context = None

        return(save_context, definition_context)


class ReproduceWorkEncoder(toml.TomlEncoder):
    def dump_str(self, v):
        """Encode a string."""
        if "\n" in v:
            return v  # If it's a multi-line string, return it as-is
        return super().dump_str(v)
    
    def dump_value(self, v):
        """Determine the type of a Python object and serialize it accordingly."""
        if isinstance(v, str) and "\n" in v:
            return '"""\n' + v.strip() + '\n' + '"""'
        return super().dump_value(v)

@requires_config
def save(content, filename, watch=True):
    """
    Save content to a file and register metadata.
    """
    # Save content to the specified file
    with open(filename, 'w') as file:
        file.write(content)

    # Capture metadata
    timestamp = datetime.datetime.now().isoformat()
    script_filename = inspect.currentframe().f_back.f_code.co_filename
    python_version = sys.version
    platform_info = platform.platform()

    # generate cryptographic hash of file contents
    content_hash = hashlib.md5(content.encode('utf-8')).hexdigest()
    timed_hash = hashlib.md5((content + timestamp).encode('utf-8')).hexdigest()
         
    save_context, definition_context = check_for_defintion_in_context(function_name='save')

    # Store metadata
    metadata = {
        "type": "file",
        "timestamp": timestamp,
        #"script_filename": script_filename,
        #"python_version": python_version,
        #"platform_info": platform_info,
        "content_hash": content_hash,
        "timed_hash": timed_hash,
        #"save_context": save_context,
        #"definition_context": definition_context
    }
    cell_index = get_cell_index()
    if cell_index:
        metadata["cell_index"] = cell_index

    base_config = read_base_config()
    #reproduce_work_watched_files = base_config['repro.files.watch']
    

    # For this demo, let's return the metadata (in practice, you might want to log it, save it to another file, etc.)
    if watch:
        update_watched_files(add=[filename])

    # check if dynamic file exists
    if not os.path.exists(Path(base_config['repro']['files']['dynamic'])):
        with open(Path(base_config['repro']['files']['dynamic']), 'w') as file:
            file.write(toml.dumps({}))

    with open(Path(base_config['repro']['files']['dynamic']), 'r') as file:
        dynamic_data = toml.load(file)
        
    dynamic_data[filename] = metadata

    with open(Path(base_config['repro']['files']['dynamic']), 'w') as file:
        toml.dump(dynamic_data, file, encoder=ReproduceWorkEncoder())

    return metadata



def reproducible(var_assignment_func):
    """
    A decorator to register the line number and timestamp when a variable is assigned.
    """
    def wrapper(value, var_name):
        # Get the current frame and line number
        frame = inspect.currentframe()
        line_number = frame.f_back.f_lineno
        
        # Get the current timestamp
        timestamp = datetime.datetime.now().isoformat()
        
        # Get the filename of the caller
        filename = frame.f_back.f_code.co_filename
        
        # Execute the variable assignment function
        var_assignment_func(var_name, value)

        #save_context, definition_context = check_for_defintion_in_context(function_name='assign')
        
        # Register the variable name, line number, timestamp, and filename
        VAR_REGISTRY[var_name] = {
            "type": "string",
            #"line_number": line_number,
            "timestamp": timestamp,
            #"filename": filename,
            #"save_context": save_context,
            #"definition_context": definition_context
        }
        cell_index = get_cell_index()
        if cell_index:
            VAR_REGISTRY[var_name]["cell_index"] = cell_index

        if type(value) is not str:
            value = str(value)
            print(f"WARNING: value of {var_name} was not a string. Converted to string: {value}.")
        
        VAR_REGISTRY[var_name]['value'] = value

        config = read_base_config()


        # check if dynamic file exists
        if not os.path.exists(Path(config['repro']['files']['dynamic'])):
            with open(Path(config['repro']['files']['dynamic']), 'w') as file:
                file.write(toml.dumps({}))
        with open(Path(config['repro']['files']['dynamic']), 'r') as file:
            dynamic_data = toml.load(file)
            
        dynamic_data[var_name] = VAR_REGISTRY[var_name]
        
        with open(Path(config['repro']['files']['dynamic']), 'w') as file:
            toml.dump(dynamic_data, file, encoder=ReproduceWorkEncoder())

        return value
    return wrapper

@reproducible
def assign(value, var_name):
    globals()[var_name] = value

# Test code


In [5]:
assign(67890, "test_var_timestamp_1")  # This should capture this line number and timestamp
assign("Hello again!", "test_var_timestamp_2")  # And this line number and timestamp

VAR_REGISTRY

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

{'test_var_timestamp_1': {'type': 'string',
  'timestamp': '2023-10-02T00:45:15.324766',
  'value': '67890'},
 'test_var_timestamp_2': {'type': 'string',
  'timestamp': '2023-10-02T00:45:15.342110',
  'value': 'Hello again!'}}

In [17]:
VAR_REGISTRY

{'test_var_timestamp_1': {'type': 'string',
  'timestamp': '2023-10-02T00:42:18.461407',
  'value': '67890'},
 'test_var_timestamp_2': {'type': 'string',
  'timestamp': '2023-10-02T00:42:18.472973',
  'value': 'Hello again!'}}

In [20]:
config = read_base_config()
print(toml.dumps(VAR_REGISTRY))

[test_var_timestamp_1]
type = "string"
timestamp = "2023-10-02T00:42:18.461407"
value = "67890"

[test_var_timestamp_2]
type = "string"
timestamp = "2023-10-02T00:42:18.472973"
value = "Hello again!"



In [7]:
# Test the save function
test_content = "This is a test content for the save function."

In [8]:
metadata = save(test_content, "saved_file.txt")
metadata

<IPython.core.display.Javascript object>

Updated watched files to ['reproduce/main.md', 'reproduce/data.toml', 'reproduce/latex/template.tex', 'saved_file.txt']


{'type': 'file',
 'timestamp': '2023-10-02T00:45:22.945064',
 'python_version': '3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:23:23) \n[GCC 10.3.0]',
 'platform_info': 'Linux-6.4.16-orbstack-00105-g14094bfeec09-aarch64-with-glibc2.35',
 'content_hash': 'd1866c6aa7d10eb57a35cc88a77802c5',
 'timed_hash': '701881869e4321228a9dbcf6d1b20d95'}

In [12]:
# Test
#x = 10
#y = "Hello"
#z = [1, 2, 3]
#save()

In [13]:
#| hide
import nbdev; nbdev.nbdev_export()